# <div align="center"> Generating Market Neutral Portfolios/ Portfolio without Market Risk/ Market Risk Hedged Portfolio </div>

In [1]:
#Importing Libraries
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [3]:
#Putting inputs for stock data and S&P500
tickers = tickers = ["VRSN", "ATVi", "MCO","AAPL","TSLA","SPY"]
start = dt.datetime(2020,1,1)
end = dt.datetime(2022,11,28)

In [4]:
#Loading Stock Data and calculatings its returns
Price = pd.DataFrame()
for t in tickers:
    Price[t] = yf.download(t,start,end)["Adj Close"]
Re = Price.pct_change()
Re = Re.loc[~(Re==0).all(axis=1)]
Re= Re.dropna()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
Re.head()

,VRSN,ATVi,MCO,AAPL,TSLA,SPY
Date,,,,,,
2020-01-03,0.021095,0.000341,-0.002482,-0.009722,0.029633,-0.007572
2020-01-06,0.009259,0.018238,0.003110,0.007968,0.019255,0.003815
2020-01-07,0.002318,0.010044,-0.003597,-0.004703,0.038801,-0.002812
2020-01-08,0.004626,-0.007623,0.019170,0.016086,0.049205,0.005330
2020-01-09,0.023169,-0.009018,0.013150,0.021241,-0.021945,0.006780


### Performing regression for each stock with S&P500

In [7]:
#Here, we run the regression of each stock with the market and then put the results in the dataframe 'Regression_results' 
#"Regression_result" is a dataframe containing alpha and beta of each regressed stock.

Regression_results = pd.DataFrame([], columns=["Alpha","Beta"], index=tickers[:-1])
for stock in Regression_results.index:
    temp = Re.copy()
    x= sm.add_constant(temp["SPY"])
    y = temp[str(stock)]
    results = sm.OLS(y,x).fit()
    Regression_results.loc[stock,"Alpha"] = results.params[0]
    Regression_results.loc[stock,"Beta"] = results.params[1]

In [8]:
#The alpha and beta of each stock with S&P500
Regression_results

,Alpha,Beta
VRSN,-0.00025,1.005427
ATVi,0.000306,0.524829
MCO,0.000031,1.213247
AAPL,0.00065,1.211839
TSLA,0.002851,1.489047


Alpha is the abnormal return that we get by investing in the specified stock instead of market.

Beta is how the stock is moving in respect to the market.

In [34]:
#Here, we perform the hedge of each stock with market to create a market neutral portfolio and generate the alpha from the stock
#The hedged portfolio is again regressed with the market to analyse the hedged portfolio results and how it is working.
#Hedged portfolio regression results are in the "Hedged_portfolio" dataframe which contains various other elements as well.

Hedged_portfolio = pd.DataFrame([], columns=["Alpha", "Beta","Stock Average Return","Stock Volatility","Stock Sharpe Ratio","Hedged_P Average Return","Hedged_P Volatility", "Hedged_P SR (Apraisal Ratio)"], index=tickers[:-1])

for stock in Hedged_portfolio.index:
    temp = Re.copy()
    portfolio = temp[stock]
    MKT = temp["SPY"]
    tracking = Regression_results.loc[stock,"Beta"]*MKT
    hedged = portfolio - tracking
    Hedged_portfolio.at[stock,"Hedged_P Average Return"] = hedged.mean() * 252
    Hedged_portfolio.at[stock,"Hedged_P Volatility"] = hedged.std() * (252**0.5)
    Hedged_portfolio.at[stock,"Hedged_P SR (Apraisal Ratio)"] = hedged.mean()/hedged.std() * (252**0.5)
    Hedged_portfolio.at[stock,"Stock Average Return"] = portfolio.mean() * 252
    Hedged_portfolio.at[stock,"Stock Volatility"] = portfolio.std() * (252**0.5)
    Hedged_portfolio.at[stock,"Stock Sharpe Ratio"] = temp[stock].mean()/temp[stock].std() *(252**0.5)
    x = sm.add_constant(temp["SPY"])
    y = hedged
    results = sm.OLS(y,x).fit()
    Hedged_portfolio.loc[stock,"Alpha"] = results.params[0]*252
    Hedged_portfolio.loc[stock,"Beta"] = abs(results.params[1])

In [35]:
for i in Hedged_portfolio.columns:
    Hedged_portfolio[i]=pd.to_numeric(Hedged_portfolio[i])
    Hedged_portfolio[i] = round(Hedged_portfolio[i],3)

In [36]:
Hedged_portfolio

,Alpha,Beta,Stock Average Return,Stock Volatility,Stock Sharpe Ratio,Hedged_P Average Return,Hedged_P Volatility,Hedged_P SR (Apraisal Ratio)
VRSN,-0.063,0.0,0.058,0.330,0.176,-0.063,0.211,-0.298
ATVi,0.077,0.0,0.140,0.341,0.412,0.077,0.314,0.245
MCO,0.008,0.0,0.154,0.374,0.412,0.008,0.216,0.036
AAPL,0.164,0.0,0.310,0.370,0.837,0.164,0.210,0.779
TSLA,0.718,0.0,0.898,0.720,1.248,0.718,0.615,1.169


### <span style='color:green'> • As we can see, Beta = 0 , which means it is a market neutral portfolio. <br> • If we invest in this hedged portfolio, we can earn the alpha of this stock <br> • Apprial Ratio tells the effectiveness of investing in a hedged portfolio to generate the given alpha <br> • Also, the alpha of the portfolio should be equal to hedged portfolio average returns as shown by the table as well <br> • The volatility and avg returns are lesser in the hedged portfolio as we have eliminated the effect of market (systematic risk) from the portfolio. <br> </span>